In [1]:
import sqlite3
import pandas as pd

# Create a connection to a new database in memory
conn = sqlite3.connect('company_data.db')

# Let's create a 'Sales' table to start
query = """
CREATE TABLE IF NOT EXISTS sales (
    order_id INTEGER PRIMARY KEY,
    product_name TEXT,
    category TEXT,
    price REAL,
    quantity INTEGER,
    order_date DATE
);
"""
conn.execute(query)

# Add some sample data for us to play with
seed_data = [
    (101, 'iPhone 15', 'Electronics', 999.00, 2, '2023-10-01'),
    (102, 'Yoga Mat', 'Fitness', 25.50, 10, '2023-10-02'),
    (103, 'Coffee Maker', 'Home', 45.00, 5, '2023-10-02'),
    (104, 'Running Shoes', 'Fitness', 80.00, 3, '2023-10-03'),
    (105, 'Laptop', 'Electronics', 1200.00, 1, '2023-10-04')
]

conn.executemany("INSERT INTO sales VALUES (?, ?, ?, ?, ?, ?)", seed_data)
conn.commit()

print("Database ready! Your first SQL table 'sales' is created.")

Database ready! Your first SQL table 'sales' is created.


In [2]:
# Standard SQL Query
query = "SELECT product_name, category FROM sales WHERE category = 'Fitness'"

# Using Pandas to view the SQL result cleanly
df_result = pd.read_sql_query(query, conn)
display(df_result)

,product_name,category
0,Yoga Mat,Fitness
1,Running Shoes,Fitness


In [3]:
# The Business Question: High-value Electronics
query = """
SELECT product_name, price, quantity
FROM sales
WHERE category = 'Electronics' AND price >= 1000
ORDER BY price DESC;
"""

df_electronics = pd.read_sql_query(query, conn)
display(df_electronics)

,product_name,price,quantity
0,Laptop,1200.0,1


In [4]:
# The Business Question: Revenue by Category
query = """
SELECT category, SUM(price * quantity) AS total_revenue
FROM sales
GROUP BY category
ORDER BY total_revenue DESC;
"""

df_revenue = pd.read_sql_query(query, conn)
display(df_revenue)

,category,total_revenue
0,Electronics,3198.0
1,Fitness,495.0
2,Home,225.0


In [5]:
# Create the Customers table
conn.execute("""
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT,
    city TEXT
);
""")

# Add some customer data
customer_data = [
    (1, 'Alice Gupta', 'Mumbai'),
    (2, 'Rahul Sharma', 'Delhi'),
    (3, 'Sanya Iyer', 'Bangalore')
]

# Update our sales table to include a customer_id column (linking the data)
conn.execute("ALTER TABLE sales ADD COLUMN customer_id INTEGER;")

# Map our existing orders to these customers
conn.execute("UPDATE sales SET customer_id = 1 WHERE order_id IN (101, 105);")
conn.execute("UPDATE sales SET customer_id = 2 WHERE order_id IN (102, 103, 104);")

conn.executemany("INSERT INTO customers VALUES (?, ?, ?)", customer_data)
conn.commit()

print("Customers table created and linked!")

Customers table created and linked!


In [6]:
query = """
SELECT customers.customer_name, sales.product_name, sales.price
FROM sales
JOIN customers ON sales.customer_id = customers.customer_id;
"""

df_joined = pd.read_sql_query(query, conn)
display(df_joined)

,customer_name,product_name,price
0,Alice Gupta,iPhone 15,999.0
1,Rahul Sharma,Yoga Mat,25.5
2,Rahul Sharma,Coffee Maker,45.0
3,Rahul Sharma,Running Shoes,80.0
4,Alice Gupta,Laptop,1200.0


In [7]:
# SQL Query: Revenue by City
query = """
SELECT customers.city, SUM(sales.price * sales.quantity) AS total_revenue
FROM sales
JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.city
ORDER BY total_revenue DESC;
"""

# Fetch the result into Python
city_revenue = pd.read_sql_query(query, conn)
display(city_revenue)

,city,total_revenue
0,Mumbai,3198.0
1,Delhi,720.0


In [8]:
# Adding a new customer who has no orders
conn.execute("INSERT INTO customers (customer_id, customer_name, city) VALUES (4, 'Vikram Singh', 'Pune');")
conn.commit()

In [9]:
query = """
SELECT customers.customer_name, customers.city, sales.order_id
FROM customers
LEFT JOIN sales ON customers.customer_id = sales.customer_id
WHERE sales.order_id IS NULL;
"""

ghost_customers = pd.read_sql_query(query, conn)
display(ghost_customers)

,customer_name,city,order_id
0,Sanya Iyer,Bangalore,None
1,Vikram Singh,Pune,None


In [10]:
# Cleaning data in the query itself
query = """
SELECT
    UPPER(product_name) AS product_clean,
    LOWER(category) AS category_clean,
    ROUND(price, 2) AS price_formatted,
    ROUND(price * quantity, 2) AS line_total
FROM sales;
"""

df_cleaned = pd.read_sql_query(query, conn)
display(df_cleaned)

,product_clean,category_clean,price_formatted,line_total
0,IPHONE 15,electronics,999.0,1998.0
1,YOGA MAT,fitness,25.5,255.0
2,COFFEE MAKER,home,45.0,225.0
3,RUNNING SHOES,fitness,80.0,240.0
4,LAPTOP,electronics,1200.0,1200.0


In [11]:
query = """
SELECT
    product_name,
    price * quantity AS total,
    CASE
        WHEN (price * quantity) > 1000 THEN 'VIP Order'
        WHEN (price * quantity) > 500 THEN 'High Value'
        ELSE 'Standard'
    END AS order_priority
FROM sales;
"""

df_priority = pd.read_sql_query(query, conn)
display(df_priority)

,product_name,total,order_priority
0,iPhone 15,1998.0,VIP Order
1,Yoga Mat,255.0,Standard
2,Coffee Maker,225.0,Standard
3,Running Shoes,240.0,Standard
4,Laptop,1200.0,VIP Order


In [12]:
# The Master Operations Dashboard Query
query = """
SELECT
    c.customer_name,
    c.city,
    UPPER(s.product_name) AS product,
    ROUND(s.price * s.quantity, 2) AS total_order_value,
    CASE
        WHEN (s.price * s.quantity) >= 1000 THEN '🔥 Urgent / VIP'
        WHEN (s.price * s.quantity) >= 500 THEN '⭐ High Priority'
        ELSE 'Standard'
    END AS shipping_priority
FROM sales AS s
JOIN customers AS c ON s.customer_id = c.customer_id
ORDER BY total_order_value DESC;
"""

# Execute and display in a professional format
dashboard_df = pd.read_sql_query(query, conn)
print("--- LOGISTICS & OPERATIONS DASHBOARD ---")
display(dashboard_df)

--- LOGISTICS & OPERATIONS DASHBOARD ---


,customer_name,city,product,total_order_value,shipping_priority
0,Alice Gupta,Mumbai,IPHONE 15,1998.0,🔥 Urgent / VIP
1,Alice Gupta,Mumbai,LAPTOP,1200.0,🔥 Urgent / VIP
2,Rahul Sharma,Delhi,YOGA MAT,255.0,Standard
3,Rahul Sharma,Delhi,RUNNING SHOES,240.0,Standard
4,Rahul Sharma,Delhi,COFFEE MAKER,225.0,Standard
